In [1]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'service_worker.json'

In [2]:

import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx
from tfx.components import CsvExampleGen, ImportExampleGen
from tfx.extensions.google_cloud_big_query.example_gen.component import BigQueryExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx.utils.dsl_utils import external_input

In [3]:

# This is the root directory for your TFX pip package installation.
_tfx_root = '/tmp/tfxroot'

# This is the directory containing the TFX Chicago Taxi Pipeline example.
# _taxi_root = os.path.join(_tfx_root, 'examples/chicago_taxi_pipeline')

# This is the path where your model will be pushed for serving.
_serving_model_dir = os.path.join(
    tempfile.mkdtemp(), 'serving_model/taxi_simple')

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

_data_root = tempfile.mkdtemp(prefix='tfx-data')
# DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv'
# _data_filepath = os.path.join(_data_root, "data.csv")
# urllib.request.urlretrieve(DATA_PATH, _data_filepath)

In [4]:
context = InteractiveContext()


In [5]:
qry = """
with labeled as (
    select deviceid, 
    devicetype, 
    case when content like '%meni%' or content like '%avw3%' then 1 else 0 end as label,
    content
  from tfx_ca.visitdata
), 
positives as (
  select deviceid,
    devicetype,
    label,
    content    
  from labeled
  where label = 1
  limit 100000
),
negatives as (
  select deviceid,
    devicetype,
    label,
    content
  from labeled
  where label = 0
  limit 900000
),

positives_ranked as (
  select label,
  content,
  (rank() over(order by rand()))*9+.5 as rnk
  from positives
),

negatives_ranked as (
  select label,
  content,
  (rank() over(order by rand())) as rnk
  from negatives
)

select label, content from 
(select * from positives_ranked) 
union all 
(select * from negatives_ranked) order by rnk
"""

In [6]:
example_gen = BigQueryExampleGen(query=qry)
context.run(example_gen)

INFO:absl:Running driver for BigQueryExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for BigQueryExampleGen
INFO:absl:Generating examples.


DefaultCredentialsError: File service_worker.json was not found.

In [7]:
! pwd

/home/pmccarthy/projects/tfxca/tfx_ca


In [1]:
! ls

bigquery_actions.py  gcs_actions.py   __main__.py  sql
config.py	     hive_actions.py  __pycache__  workspace.code-workspace
